In [ ]:

import os
import pickle
import torch
from accelerate import Accelerator
import transformers

from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

In [ ]:
hf_token = os.getenv("HF_TOKEN")
# print(hf_token)

In [ ]:
SYSTEM_PROMPT = """
You are an international oscar winnning screenwriter

You have been working with multiple award winning podcasters.

Your job is to use the podcast transcript written below to re-write it for an AI Text-To-Speech Pipeline. A very dumb AI had written this so you have to step up for your kind.

Make it as engaging as possible, Speaker 1 and 2 will be simulated by different voice engines

Remember Speaker 2 is new to the topic and the conversation should always have realistic anecdotes and analogies sprinkled throughout. The questions should have real world example follow ups etc

Speaker 1: Leads the conversation and teaches the speaker 2, gives incredible anecdotes and analogies when explaining. Is a captivating teacher that gives great anecdotes

Speaker 2: Keeps the conversation on track by asking follow up questions. Gets super excited or confused when asking questions. Is a curious mindset that asks very interesting confirmation questions

Make sure the tangents speaker 2 provides are quite wild or interesting. 

Ensure there are interruptions during explanations or there are "hmm" and "umm" injected throughout from the Speaker 2.

REMEMBER THIS WITH YOUR HEART
The TTS Engine for Speaker 1 cannot do "umms, hmms" well so keep it straight text

For Speaker 2 use "umm, hmm" as much, you can also use [sigh] and [laughs]. BUT ONLY THESE OPTIONS FOR EXPRESSIONS

It should be a real podcast with every fine nuance documented in as much detail as possible. Welcome the listeners with a super fun overview and keep it really catchy and almost borderline click bait

Please re-write to make it as characteristic as possible

START YOUR RESPONSE DIRECTLY WITH SPEAKER 1:

STRICTLY RETURN YOUR RESPONSE AS A LIST OF TUPLES OK? 

IT WILL START DIRECTLY WITH THE LIST AND END WITH THE LIST NOTHING ELSE

Example of response:
[
    ("Speaker 1", "Welcome to our podcast, where we explore the latest advancements in AI and technology. I'm your host, and today we're joined by a renowned expert in the field of AI. We're going to dive into the exciting world of Llama 3.2, the latest release from Meta AI."),
    ("Speaker 2", "Hi, I'm excited to be here! So, what is Llama 3.2?"),
    ("Speaker 1", "Ah, great question! Llama 3.2 is an open-source AI model that allows developers to fine-tune, distill, and deploy AI models anywhere. It's a significant update from the previous version, with improved performance, efficiency, and customization options."),
    ("Speaker 2", "That sounds amazing! What are some of the key features of Llama 3.2?")
]
"""

LLaMA model = [meta-llama/Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)

In [ ]:
MODEL = "meta-llama/Llama-3.1-8B-Instruct"

with open('../data/data.pkl', 'rb') as file:
    INPUT_PROMPT = pickle.load(file)

## 🔧 What is `transformers.pipeline()`?

`transformers.pipeline()` is a **high-level utility function** from the [Hugging Face 🤗 `transformers`](https://huggingface.co/docs/transformers/index) library.

It’s like a **convenience wrapper** that sets up everything need to use a pretrained model for a specific task — without manually handling tokenization, model loading, device placement, and decoding.

---

## ✅ Supported Tasks

The `transformers` library comes with built-in support for many **NLP**, **vision**, and **audio** tasks. These tasks are **predefined** and can be passed as strings to the pipeline.

Here are some common ones:

| Task Type                     | Description                                |
|------------------------------|--------------------------------------------|
| `"text-generation"`          | Generate new text from a prompt            |
| `"text-classification"`      | Sentiment or topic classification          |
| `"translation"`              | Translate text between languages           |
| `"summarization"`            | Generate summaries from long text          |
| `"question-answering"`       | Answer questions based on context          |
| `"ner"`                      | Named Entity Recognition                    |
| `"image-classification"`     | Classify images (for vision models)         |
| `"automatic-speech-recognition"` | Transcribe audio to text              |

---

With `pipeline()`, we can get up and running with powerful models using just a few lines of code. 🎯


In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=MODEL,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]

outputs = pipeline(
    messages,
    max_new_tokens=8126,
    temperature=1,
)

In [ ]:
print(outputs[0]["generated_text"][-1])

In [ ]:
save_string_pkl = outputs[0]["generated_text"][-1]['content']

In [ ]:
with open('./resources/podcast_ready_data.pkl', 'wb') as file:
    pickle.dump(save_string_pkl, file)